# SQL

### Цель:
    Сделать исследовательский анализ данных потенциальной компании-цели, благодаря которому станет возможно сформулировать ценностное предложение для нового продукта
    
#### Задачи:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
- Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок.

## 1. Подключение к базе данных и изучение содержащихся в ней таблиц


In [3]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
             'pwd': 'Sdf4$2;d-d30pp', # пароль
             'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
             'port': 6432, # порт подключения
             'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(db_config['user'],
                 db_config['pwd'],
                 db_config['host'],
                 db_config['port'],
                 db_config['db'])

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'}) 

In [5]:
def SQL_request_result(query):
    """ Функция, выводящая результат SQL-запроса
    
    принимает на вход SQL-запрос
    """
    
    return pd.io.sql.read_sql(query, con = engine)

### 1.1. Изучим таблицу books

In [33]:
# формируем запрос
query = """SELECT *
           FROM books
           LIMIT 5;
        """

# выводим результат
SQL_request_result(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


**Описание данных**
Содержит данные о книгах:

- book_id — идентификатор книги
- author_id — идентификатор автора
- title — название книги
- num_pages — количество страниц
- publication_date — дата публикации книги
- publisher_id — идентификатор издателя

### 1.2. Изучим таблицу authors

In [34]:
# формируем запрос
query = """SELECT *
           FROM authors
           LIMIT 5;
        """

# выводим результат
SQL_request_result(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


#### Описание данных
Содержит данные об авторах:

- author_id — идентификатор автора
- author — имя автора

### 1.3. Изучим таблицу publishers

In [35]:
# формируем запрос
query = """SELECT *
           FROM publishers
           LIMIT 5;
        """

# выводим результат
SQL_request_result(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Описание данных
Содержит данные об издательствах:

publisher_id — идентификатор издательства
publisher — название издательства

### 1.4. Изучим таблицу ratings

In [36]:
# формируем запрос
query = """SELECT *
           FROM ratings
           LIMIT 5;
        """

# выводим результат
SQL_request_result(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


**Описание данных**
Содержит данные о пользовательских оценках книг:

- rating_id — идентификатор оценки
- book_id — идентификатор книги
- username — имя пользователя, оставившего оценку
- rating — оценка книги

### 1.5. Изучим таблицу reviews

In [37]:
# формируем запрос
query = """SELECT *
           FROM reviews
           LIMIT 5;
        """

# выводим результат
SQL_request_result(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


**Описание данных**
Содержит данные о пользовательских обзорах на книги:

review_id — идентификатор обзора
book_id — идентификатор книги
username — имя пользователя, написавшего обзор
text — текст обзора

## 2. Анализ данных

### 2.1. Сколько книг вышло после 1 января 2000 года


In [38]:
# формируем запрос
query = """SELECT COUNT(book_id) AS number_of_books_after_date
           FROM books
           WHERE publication_date > '2000-01-01';
        """

# выводим результат
SQL_request_result(query)

,number_of_books_after_date
0,819


**Вывод:** Количество книг, выпущенных после 1 января 2000 года - 819 из 1 000. Относительно всей базы данных это кол-во существенное, но в абсолютных величинах это небольшое число книг. Необходимо существенно увеличить кол-во новых книг, чтобы оттянуть клиентов у конкурентов 

### 2.2. Количество обзоров и средняя оценку по каждой книге


In [19]:

# формируем запрос
query = """SELECT books.title AS title,
                  SUBQ1.number_of_reviews,
                  ROUND(SUBQ2.average_rating, 2) AS average_rating
           FROM books
           INNER JOIN 
                (SELECT book_id,
                        COUNT(review_id) as number_of_reviews
                 FROM reviews
                 GROUP BY book_id) AS SUBQ1
                                         ON SUBQ1.book_id = books.book_id
           INNER JOIN 
                (SELECT book_id,
                        AVG(rating) as average_rating
                 FROM ratings
                 GROUP BY book_id) AS SUBQ2
                                         ON SUBQ2.book_id = books.book_id
           GROUP BY title,
                    SUBQ1.number_of_reviews,
                    average_rating
           ORDER BY SUBQ1.number_of_reviews DESC
           LIMIT 10;
        """

In [20]:
#Ограничим отображение топ-10 книгами по числу пользовательских обзоров
# выводим результат
SQL_request_result(query)

,title,number_of_reviews,average_rating
0,Twilight (Twilight #1),7,3.66
1,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,The Da Vinci Code (Robert Langdon #2),6,3.83
3,The Lightning Thief (Percy Jackson and the Oly...,6,4.08
4,Outlander (Outlander #1),6,4.13
5,Eat Pray Love,6,3.40
6,Water for Elephants,6,3.98
7,The Hobbit or There and Back Again,6,4.13
8,The Book Thief,6,4.26
9,The Alchemist,6,3.79


**Вывод:**
- Больщое кол-во отзывов не является следствием высоких оценок для книг. В топ-10 книг по кол-ву отзывов в основном фантастика.

- Людям не нравится свое реальность и они бегут в книжки. Нужно сосредоточиться на фантастике при запуске. Но также карантин нам показал, что это время развиваться, поэтому стоит добавлять книги по личностному росту

### 2.3. Издательство, которое выпустило наибольшее число книг толще 50 страниц (исключаем брошуры)


Книги менее 50 страниц - брошюры. Исключаем их из анализа



In [41]:
# формируем запрос
query = """SELECT publishers.publisher AS publisher_name,
                  COUNT (SUBQ.book_id) AS number_of_books
           FROM (
               SELECT book_id,
                      publisher_id
               FROM books
               WHERE num_pages > 50) AS SUBQ
           INNER JOIN publishers ON publishers.publisher_id = SUBQ.publisher_id
           GROUP BY publisher_name
           ORDER BY number_of_books DESC
           LIMIT 1;
        """

In [42]:
# Выводим на экран издательство, выпустившее наибольшее число книг толще 50 страниц и количество выпущенных им книг

SQL_request_result(query)


,publisher_name,number_of_books
0,Penguin Books,42


**Вывод:**
- В базе данных самое часто встречающееся издательство - "Penguin Books"
- Нужно обязательно продолжать работать с ним после тщательного анализа его продукции

### 2.4. Определить автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками


In [45]:
# Учитываются только книги с 50 и более пользовательскими оценками

# формируем запрос
query = """SELECT authors.author AS author_name,
                  ROUND(AVG(SUBQ.average_rating), 2) AS avg_rate    
           FROM books
           LEFT JOIN (
                   SELECT book_id,
                          COUNT(rating) AS cnt,
                          AVG(rating) AS average_rating
                   FROM ratings
                   GROUP BY book_id
                   ) AS SUBQ
                ON SUBQ.book_id = books.book_id
           LEFT JOIN authors ON authors.author_id = books.author_id
           WHERE SUBQ.cnt >= 50
           GROUP BY author_name
           ORDER BY avg_rate DESC
           LIMIT 5;   
        """

In [46]:
# выводим результат
SQL_request_result(query)


,author_name,avg_rate
0,J.K. Rowling/Mary GrandPré,4.28
1,Markus Zusak/Cao Xuân Việt Khương,4.26
2,J.R.R. Tolkien,4.26
3,Louisa May Alcott,4.19
4,Rick Riordan,4.08


**Вывод:**
- Топ-3 авторов по средней оценки примерно равны и опять же преобладают фантасты. 
- Еще одно подтверждение, что в продукт стоит добавить книги с фантастической тематикой и авторов, которая выпустили наиболее популярные книги в этом жанре



### 2.5. Посчитайть среднее количество обзоров от пользователей, которые поставили больше 50 оценок

In [47]:
# формируем запрос
query = """SELECT ROUND(AVG(SUBQ2.review_cnt), 0) AS average_number_of_reviews
           FROM (
                SELECT username  
                FROM ratings
                GROUP BY username
                HAVING COUNT(rating_id) > 50) AS SUBQ1
           LEFT JOIN (
                       SELECT username,
                              COUNT(review_id) AS review_cnt
                       FROM reviews
                       GROUP BY username) AS SUBQ2
            ON SUBQ2.username = SUBQ1.username
        """

In [48]:
# выводим результат
SQL_request_result(query)

,average_number_of_reviews
0,24.0


**Вывод**
- 24 обзора на пользователя - хороший показатель, учитывая, что некоторые люди столько читают за всю жизнь. 
- Систему поощрения пользователей за обзоры может наращивать базу обзоров, а это в свою очередь будет помогать остальным пользователям подбирать интересные для себя книги

## 3. Общие выводы
- Необходимо нарашивать базу новых книг, чтобы пользователем было интересно возвращаться за новой книгой в наш продукт. 
- В топ-10 самых оцениваемых книг и жанров входит преимущественно фэнтези. Наивысшая средняя оценка у авторов книг в этом жанре.
- Проанализировать книги крупных издателей на предмет их качества и возможно нужно отказаться от крупных издательств в пользу более мелких, но с высокими рейтингами читаемости. 
- Стоит предусмотреть систему вознаграждений пользователей за оставленные отзывы о книгах.